In [1]:
days = 20*5

DemandC1 = [1600 1600 1600 1800 1800 0]

Mfactor = 60;
C11factor = 4;

InvC1 = zeros(days+50,1)
InvM = zeros(days+50,1)
InvC11 = zeros(days+50,1)

InvM[1:end] = 20;
InvC11[1:end] = 5;

cycleC1 = 1703

t=1;
demandMonth = 1
nOrders=0
noOrder = true
while t < days
    if t >= (demandMonth*20)
        InvC1[t:end] -= DemandC1[demandMonth]
        demandMonth += 1
    end
    if InvC1[t] > DemandC1[demandMonth]
        keepProduction = false
    else
        keepProduction = true
    end

    Mleft = (DemandC1[demandMonth] - InvC1[t]) * Mfactor
    C11left = (DemandC1[demandMonth] - InvC1[t]) * C11factor
    if ((InvM[t] < Mleft) || (InvC11[t] < C11left)) && noOrder && keepProduction
        nOrders+=1
        tLast = t
        tAdd = round(Int64,2*randn(1)[1] + 5)
        if tAdd < 0
            t+=0
        else
            t+=tAdd
        end
        
        InvM[tLast:t] = InvM[tLast]
        InvC11[tLast:t] = InvC11[tLast]
        InvC1[tLast:t] = InvC1[tLast]
        
        InvM[t:end] += max(DemandC1[demandMonth]* Mfactor, cycleC1 * Mfactor * 7)
        InvC11[t:end] += max(DemandC1[demandMonth]* C11factor, cycleC1 * C11factor * 7)
    else
        if InvC1[t] > DemandC1[demandMonth]
            keepProduction = false
        else
            keepProduction = true
        end

        while ((InvM[t]-Mfactor)>0) && ((InvC11[t]-C11factor)>0) && ((t+1)<= days) && keepProduction          
            prod = round(Int64, 200*randn(1)[1] + cycleC1/2)
            nM = div(InvM[t], Mfactor)
            nC11 = div(InvC11[t], C11factor)
            
            nC1 = min(nM, nC11, prod)
            
            InvC1[t:end] += nC1
            InvM[t:end] -= nC1*Mfactor
            InvC11[t:end] -= nC1*C11factor            
            defect = rand()/10
            
            InvC1[t:end] -= round(Int64, InvC1[t+1]*defect)
            if InvC1[t] > DemandC1[demandMonth]
                break
            end
            t+=1
            if t >= (demandMonth*20)
                InvC1[t:end] -= DemandC1[demandMonth]
                demandMonth += 1
            end
            Mleft = (DemandC1[demandMonth] - InvC1[t]) * Mfactor
            C11left = (DemandC1[demandMonth] - InvC1[t]) * C11factor
            if ((InvM[t] < Mleft) || (InvC11[t] < C11left)) && noOrder
                noOrder = false
                nOrders += 1
                tLast = t
                tAdd = round(Int64,2*randn(1)[1] + 5)
                if tAdd < 0
                    tOrder=t
                else
                    tOrder=tAdd+t
                end

                InvM[tLast:tOrder] = InvM[tLast]
                InvC11[tLast:tOrder] = InvC11[tLast]
                InvC1[tLast:tOrder] = InvC1[tLast]
            end
            if !noOrder
                if !(((InvM[t]-Mfactor)>0) && ((InvC11[t]-C11factor)>0) && ((t+1)<= days) && keepProduction)
                    t = tOrder
                    noOrder = true
                    InvM[t:end] += max(Mleft* Mfactor, cycleC1 * Mfactor * 7)
                    InvC11[t:end] += max(C11left* C11factor, cycleC1 * C11factor * 7)
                end
                
            end
        end
    end
    t+=1
end

nOrders

1

In [2]:
using Plots
plot(1:(days+50), InvM/Mfactor, label = "M scaled",title="Inventory over 5 months")
plot!(1:(days+50), InvC11/C11factor, label="C11 scaled")
plot!(1:(days+50), InvC1, label="C1 actual")

using Images
using FileIO
savefig("c1invall.png")


[Plots.jl] Initializing backend: pyplot


In [3]:
plot(1:(days+50), InvC1, label="C1", title="Inventory of C1")
using Plots
using Images
using FileIO
savefig("c1inv.png")